In [1]:
import torch 

In [2]:
torch.version

<module 'torch.version' from '/home/lab/anaconda3/envs/nsvqa/lib/python3.6/site-packages/torch/version.py'>

In [3]:
import sys
sys.path.append('/home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/')
from dataloaders.vcr import VCR, VCRLoader

/home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/data/vcr1images


In [4]:
"""
Training script. Should be pretty adaptable to whatever.
"""
import argparse
import os
import shutil

import multiprocessing
import numpy as np
import pandas as pd
import torch
from allennlp.common.params import Params
from allennlp.training.learning_rate_schedulers import LearningRateScheduler
from allennlp.training.optimizers import Optimizer
from torch.nn import DataParallel
from torch.nn.modules import BatchNorm2d
from tqdm import tqdm

from dataloaders.vcr import VCR, VCRLoader
from utils.pytorch_misc import time_batch, save_checkpoint, clip_grad_norm, \
    restore_checkpoint, print_para, restore_best_checkpoint

import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s', level=logging.DEBUG)

# This is needed to make the imports work
from allennlp.models import Model
import models
# import sys
# sys.path.append("/home/lab/A305/yifan/vqa/vqa-exp/r2c/")

In [5]:
#################################
#################################
######## Data loading stuff
#################################
#################################

parser = argparse.ArgumentParser(description='train')
parser.add_argument(
    '-params',
    dest='params',
    help='Params location',
    type=str,
)
parser.add_argument(
    '-rationale',
    action="store_true",
    help='use rationale',
)
parser.add_argument(
    '-folder',
    dest='folder',
    help='folder location',
    type=str,
)
parser.add_argument(
    '-no_tqdm',
    dest='no_tqdm',
    action='store_true',
)

_StoreTrueAction(option_strings=['-no_tqdm'], dest='no_tqdm', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

In [6]:
args = parser.parse_args()
args.params = "multiatt_scene/default.json"
args.folder = "saves/scene_r2c/flagship_answer"
params = Params.from_file(args.params)
param=params['model']
print(param['type'])

MultiAttentionQA_Scene


In [11]:
train, val, test = VCR.splits(mode='rationale' if args.rationale else 'answer',
                              embs_to_load=params['dataset_reader'].get('embs', 'bert_da'),
                              only_use_relevant_dets=params['dataset_reader'].get('only_use_relevant_dets', True))
NUM_GPUS = torch.cuda.device_count()
NUM_CPUS = multiprocessing.cpu_count()
if NUM_GPUS == 0:
    raise ValueError("you need gpus!")

def _to_gpu(td):
    if NUM_GPUS > 1:
        return td
    for k in td:
        if k != 'metadata':
            td[k] = {k2: v.cuda(non_blocking=True) for k2, v in td[k].items()} if isinstance(td[k], dict) else td[k].cuda(
                non_blocking=True)
    return td
num_workers = (4 * NUM_GPUS if NUM_CPUS >= 32 else 2*NUM_GPUS)-1
#print(f"Using {num_workers} workers out of {NUM_CPUS} possible", flush=True)
# print(f"Using {num_workers} workers out of {NUM_CPUS} possible")

Only relevant dets
Loading embeddings from /home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/data/bert_da_answer_train.h5
Only relevant dets
Loading embeddings from /home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/data/bert_da_answer_val.h5
Only relevant dets
Loading embeddings from /home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/data/bert_da_answer_test.h5


In [12]:
loader_params = {'batch_size': 96 // NUM_GPUS, 'num_gpus':NUM_GPUS, 'num_workers':num_workers}
train_loader = VCRLoader.from_dataset(train, **loader_params)
val_loader = VCRLoader.from_dataset(val, **loader_params)
test_loader = VCRLoader.from_dataset(test, **loader_params)

ARGS_RESET_EVERY = 100
#print("Loading {} for {}".format(params['model'].get('type', 'WTF?'), 'rationales' if args.rationale else 'answer'), flush=True)
print("Loading {} for {}".format(params['model'].get('type', 'WTF?'), 'rationales' if args.rationale else 'answer'))


Loading WTF? for answer


In [13]:
model = Model.from_params(vocab=train.vocab, params=params['model'])

03/26/2019 15:00:11 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.models.model.Model'> from params {} and extras {'vocab': <allennlp.data.vocabulary.Vocabulary object at 0x7f9793caa5f8>}


ConfigurationError: 'key "type" is required at location "model."'

In [12]:
for submodule in model.detector.backbone.modules():
    if isinstance(submodule, BatchNorm2d):
        submodule.track_running_stats = False
    for p in submodule.parameters():
        p.requires_grad = False